In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import models
import emergent_constraint as ec

In [3]:
model_input_lists = []
for c in ['ocean_constraint', 'total_constraint']:
    for key in ['model_tracer_list',]: 
        model_input_lists.extend([t[0] for t in ec.get_model_tracer_lists(c)[key]])

model_list = sorted(list(set(model_input_lists)))
model_list

['CAMSv20r1',
 'CT2017',
 'CT2019B',
 'CTE2018',
 'CTE2020',
 'MIROC',
 'TM5-Flux-m0f',
 'TM5-Flux-mmf',
 'TM5-Flux-mrf',
 'TM5-Flux-mwf',
 's99oc_ADJocI40S_v2020',
 's99oc_SOCCOM_v2020',
 's99oc_v2020']

In [4]:
from ncar_jobqueue import NCARCluster
try:
    cluster
except:
    cluster = NCARCluster(memory='100GB')
    cluster.scale(32)
cluster

/glade/work/mclong/miniconda3/envs/so-co2/lib/python3.7/site-packages/distributed/node.py:161: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34508 instead
  f"Port {expected} is already in use.\n"


In [5]:
from dask.distributed import Client
try: 
    client
except: 
    client = Client(cluster) # Connect this local process to remote workers
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/proxy/34508/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/proxy/34508/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.46:38888,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/proxy/34508/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [6]:
model_objs = {}
for name in model_list:
    model_objs[name] = models.Model(name)
model_objs    

{'CAMSv20r1': <models.core.Model at 0x2b41103c4cd0>,
 'CT2017': <models.core.Model at 0x2b41103c4e50>,
 'CT2019B': <models.core.Model at 0x2b41103c4310>,
 'CTE2018': <models.core.Model at 0x2b41103c4e10>,
 'CTE2020': <models.core.Model at 0x2b41103c4e90>,
 'MIROC': <models.core.Model at 0x2b41103c4c50>,
 'TM5-Flux-m0f': <models.core.Model at 0x2b41103c4b90>,
 'TM5-Flux-mmf': <models.core.Model at 0x2b41103c43d0>,
 'TM5-Flux-mrf': <models.core.Model at 0x2b41103c4050>,
 'TM5-Flux-mwf': <models.core.Model at 0x2b41103c46d0>,
 's99oc_ADJocI40S_v2020': <models.core.Model at 0x2b41103c45d0>,
 's99oc_SOCCOM_v2020': <models.core.Model at 0x2b41103c4810>,
 's99oc_v2020': <models.core.Model at 0x2b41103c4d50>}

In [7]:
lat_ranges = [
    (-90., -30.), (-90., -35.), (-90., -40), (-90., -45.), (-90., -50.), (-90., -55.), 

]
lat_ranges = [(-90., x) for x in np.arange(-75., -14., 1)] + [(-45., -15.), (-45., -30.)]

In [8]:
%%time
dsets = {}
for name in model_list:
    print(name)    
    for lat_range in lat_ranges:
        dsets[name] = model_objs[name].open_derived_dataset(
                    'flux_ts_monthly', 
                    lat_range=lat_range,
                    persist=True,
                    clobber=False,
                    return_none=True,
                )

CAMSv20r1
CT2017
CT2019B
CTE2018
CTE2020
MIROC
TM5-Flux-m0f
TM5-Flux-mmf
TM5-Flux-mrf
TM5-Flux-mwf
s99oc_ADJocI40S_v2020
s99oc_SOCCOM_v2020
s99oc_v2020
CPU times: user 48.8 ms, sys: 3.72 ms, total: 52.5 ms
Wall time: 52.4 ms


In [9]:
%%time
dsets = {}
for name in model_list:
    print(name)    
    dsets[name] = model_objs[name].open_derived_dataset(
                'fluxes_za_ts_monthly', 
                persist=True,
                clobber=False,
                return_none=True,
            )

CAMSv20r1
CT2017
CT2019B
CTE2018
CTE2020
MIROC
TM5-Flux-m0f
TM5-Flux-mmf
TM5-Flux-mrf
TM5-Flux-mwf
s99oc_ADJocI40S_v2020
s99oc_SOCCOM_v2020
s99oc_v2020
CPU times: user 902 µs, sys: 0 ns, total: 902 µs
Wall time: 907 µs


In [11]:
for model in model_list:
    if model in ['CT2019B']:
        model_objs[model].open_derived_dataset(
            'molefractions_surface_daily',
            persist=True,
            clobber=False,            
            return_none=True,                    
        )
        model_objs[model].open_derived_dataset(
            'spo_ts_daily',            
            persist=True,
            clobber=False,           
            return_none=True,                    
        )
        model_objs[model].open_derived_dataset(
            'molefractions_theta_bins',
            kwargs_name='SO-10K-bins-300K_275K',
            lat_bounds=(-80., -45.),
            theta_bins=[(295., 305.), (270., 280.),],
            persist=True,            
            clobber=False,
            return_none=True,        
        )
        model_objs[model].open_derived_dataset(
            'molefractions_theta_bins_sectors',
            kwargs_name='SO-10K-bins-300K_275K',
            lat_bounds=(-80., -45.),
            theta_bins=[(295., 305.), (270., 280.),],
            persist=True,            
            clobber=False,
            return_none=True,  
        )
        continue
        model_objs[name].open_derived_dataset(
            'molefractions_z_za', 
            persist=True,
            clobber=False,
            return_none=True,                    
        )

writing cache file: /glade/u/home/mclong/codes/so-co2-airborne-obs/so-co2-airborne-obs/models/data-cache/CT2019B/spo_ts_daily.nc


/glade/work/mclong/miniconda3/envs/so-co2/lib/python3.7/site-packages/xarray/conventions.py:449: UserWarning: Variable 'time' has datetime type and a bounds variable but time.encoding does not have units specified. The units encodings for 'time' and 'time_bnds' will be determined independently and may not be equal, counter to CF-conventions. If this is a concern, specify a units encoding for 'time' before writing to a file.
  UserWarning,
